→ 전력 수요가 얼마나 되는지, 자립 가능한지, 얼마나 외부 에너지에 의존하는지 분석 (수요-공급 밸런스)

In [47]:
import requests
import csv
import os
from collections import defaultdict
from dotenv import load_dotenv
from time import sleep
import json

load_dotenv()
API_KEY = os.getenv("API_KEY")

sido_codes = {
    "11": "서울특별시",
    "26": "부산광역시",
    "27": "대구광역시",
    "28": "인천광역시",
    "29": "광주광역시",
    "30": "대전광역시",
    "31": "울산광역시",
    "36": "세종특별자치시",
    "41": "경기도",
    "51": "강원특별자치도",
    "43": "충청북도",
    "44": "충청남도",
    "52": "전라북도",
    "46": "전라남도",
    "47": "경상북도",
    "48": "경상남도",
    "50": "제주특별자치도"
}

years = range(2012, 2025)
months = range(1, 13)

# 용도 목록
usage_types = ["가로등", "교육용", "농사용", "산업용", "일반용", "주택용", "심야"]

# key: (year, metro) → value: dict of usage types + total
aggregated = defaultdict(lambda: defaultdict(int))

for year in years:
    for metroCd, metro_name in sido_codes.items():
        for month in months:
            print(f"📡 Fetching {year}-{month:02d} | {metro_name} ...")
            url = "https://bigdata.kepco.co.kr/openapi/v1/powerUsage/contractType.do"
            params = {
                "year": str(year),
                "month": f"{month:02d}",
                "metroCd": metroCd,
                "apiKey": API_KEY,
                "returnType": "json"
            }

            try:
                response = requests.get(url, params=params)
                raw_text = response.text

                json_start_index = raw_text.find("{")
                if json_start_index != -1:
                    raw_text = raw_text[json_start_index:]

                result = json.loads(raw_text)
                if "data" not in result:
                    print(f"data 필드 없음 at {year}-{month:02d} {metro_name}")
                    continue

                for item in result["data"]:
                    if item["metro"] != metro_name:
                        continue

                    cntr = item["cntr"]
                    usage = int(item["powerUsage"])
                    key = (year, metro_name)

                    if cntr in usage_types:
                        aggregated[key][cntr] += usage
                    else:
                        aggregated[key]["기타"] += usage

                    aggregated[key]["총합"] += usage

            except Exception as e:
                print(f"요청 오류 at {year}-{month:02d} {metro_name}: {e}")

            sleep(1)

# CSV 저장
with open("연도별 지역별 전력 사용량.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    headers = ["year", "metro"] + usage_types + ["기타", "총합"]
    writer.writerow(headers)

    for (year, metro), usage_data in sorted(aggregated.items()):
        row = [year, metro]
        for usage_type in usage_types:
            row.append(usage_data.get(usage_type, 0))
        row.append(usage_data.get("기타", 0))
        row.append(usage_data.get("총합", 0))
        writer.writerow(row)

print("연도별 지역별 전력 사용량.csv 저장됨")


📡 Fetching 2012-01 | 서울특별시 ...
📡 Fetching 2012-02 | 서울특별시 ...
📡 Fetching 2012-03 | 서울특별시 ...
📡 Fetching 2012-04 | 서울특별시 ...
📡 Fetching 2012-05 | 서울특별시 ...
📡 Fetching 2012-06 | 서울특별시 ...
📡 Fetching 2012-07 | 서울특별시 ...
📡 Fetching 2012-08 | 서울특별시 ...
📡 Fetching 2012-09 | 서울특별시 ...
📡 Fetching 2012-10 | 서울특별시 ...
📡 Fetching 2012-11 | 서울특별시 ...
📡 Fetching 2012-12 | 서울특별시 ...
📡 Fetching 2012-01 | 부산광역시 ...
📡 Fetching 2012-02 | 부산광역시 ...
📡 Fetching 2012-03 | 부산광역시 ...
📡 Fetching 2012-04 | 부산광역시 ...
📡 Fetching 2012-05 | 부산광역시 ...
📡 Fetching 2012-06 | 부산광역시 ...
📡 Fetching 2012-07 | 부산광역시 ...
📡 Fetching 2012-08 | 부산광역시 ...
📡 Fetching 2012-09 | 부산광역시 ...
📡 Fetching 2012-10 | 부산광역시 ...
📡 Fetching 2012-11 | 부산광역시 ...
📡 Fetching 2012-12 | 부산광역시 ...
📡 Fetching 2012-01 | 대구광역시 ...
📡 Fetching 2012-02 | 대구광역시 ...
📡 Fetching 2012-03 | 대구광역시 ...
📡 Fetching 2012-04 | 대구광역시 ...
📡 Fetching 2012-05 | 대구광역시 ...
📡 Fetching 2012-06 | 대구광역시 ...
📡 Fetching 2012-07 | 대구광역시 ...
📡 Fetching 2012-08 | 대구광역시 ...
📡 Fetchi

In [45]:
import pandas as pd
#지역별 발전 설비
df = pd.read_csv('yearly_metro_power_usage_by_type.csv')

print(df)

    year    metro        가로등         교육용         농사용          산업용  \
0   2021      경기도  829312448  1592183245  2829853524  73276124836   
1   2021     경상남도  282840307   437222858  2644208812  19996958734   
2   2021     경상북도  305313999   650270206  2396825065  30222892756   
3   2021    광주광역시   79330973   292250054    84581630   3179924159   
4   2021    대구광역시  125511375   357768187    88016891   6434272741   
..   ...      ...        ...         ...         ...          ...   
59  2024     전라남도  235218578   349036417  4197272144  21067940683   
60  2024     전라북도   17774791    41969373   179436752    999133046   
61  2024  제주특별자치도   47379720   152831516  1482183904    685461556   
62  2024     충청남도  178329896   506401417  2697906257  36587401447   
63  2024     충청북도  182397517   346871629  1042254843  18213696354   

            일반용          주택용  심야          기타            총합  
0   31403816122  21208073651   0  2306584289  133445948115  
1    6646915861   5020940715   0   704972207   35

In [36]:
import requests
import json

url = "https://bigdata.kepco.co.kr/openapi/v1/powerUsage/contractType.do"
params = {
    "year": 2020,
    "month": f"{month:02d}",
    "metroCd": 51,
    "apiKey": API_KEY,
    "returnType": "json"
}

response = requests.get(url, params=params)
raw_text = response.text

# JSON이 중간에 섞여있을 경우, JSON 시작 부분만 추출
json_start_index = raw_text.find("{")
if json_start_index != -1:
    raw_text = raw_text[json_start_index:]

try:
    result = json.loads(raw_text)
    print(json.dumps(result, indent=4, ensure_ascii=False))  # 결과를 보기 좋게 출력
except json.JSONDecodeError as e:
    print(f"❌ JSON 디코딩 오류: {e}")
    print("응답 원문:", raw_text)


{
    "data": [
        {
            "year": "2020",
            "month": "03",
            "metro": "강원특별자치도",
            "city": "강릉시",
            "cntr": "가로등",
            "custCnt": 13829,
            "powerUsage": 2276922,
            "bill": 251842178,
            "unitCost": 110.6
        },
        {
            "year": "2020",
            "month": "03",
            "metro": "강원특별자치도",
            "city": "강릉시",
            "cntr": "교육용",
            "custCnt": 123,
            "powerUsage": 3278400,
            "bill": 354108652,
            "unitCost": 108
        },
        {
            "year": "2020",
            "month": "03",
            "metro": "강원특별자치도",
            "city": "강릉시",
            "cntr": "농사용",
            "custCnt": 6699,
            "powerUsage": 4073027,
            "bill": 201965474,
            "unitCost": 49.6
        },
        {
            "year": "2020",
            "month": "03",
            "metro": "강원특별자치도",
            "city": "강릉시",
  

내가 분석해야 할 것 
- 지역별 전력 사용량 분석
- 각 구분별 지도를 이용해서 지역 하이라이트트
- 지역별 연도별 발전량
1. 지역별 전력 소비량 및 변화 추이 분석 → 수요 규모 파악
2. 신재생 발전량 대비 소비량 비율(자립도) → 지역 에너지 자급 수준 평가
3. 자립도와 탄소 회피량 간의 관계 분석 → 지속 가능성 측정